In [3]:
!apt-get update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic

In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Load And read Data

In [42]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('data_processing').getOrCreate()
# Setting the argument inferSchema to true indicates that Spark in the background will infer the datatypes of the values in the dataset on its own
df = spark.read.csv('sample_data.csv',inferSchema=True,header=True)


In [16]:
df.columns

['ratings', 'age', 'experience', 'family', 'mobile']

In [18]:
#get the total number of records in the dataframe:
df.count()
#We have a total of 33 records in our dataframe

33

In [22]:
#shape of dataset
print((df.count(),len(df.columns)))

(33, 5)


#### Another way of viewing the columns in the dataframe is the printSchema method of spark

The nullable property indicates if the corresponding column can
assume null values (true) or not (false)

In [23]:
df.printSchema()

root
 |-- ratings: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: double (nullable = true)
 |-- family: integer (nullable = true)
 |-- mobile: string (nullable = true)



In [24]:
#fisrt few rows of dataframe
df.show(6)

+-------+---+----------+------+-------+
|ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
|      3| 32|       9.0|     3|   Vivo|
|      3| 27|      13.0|     3|  Apple|
|      4| 22|       2.5|     0|Samsung|
|      4| 37|      16.5|     4|  Apple|
|      5| 27|       9.0|     1|     MI|
|      4| 27|       9.0|     0|   Oppo|
+-------+---+----------+------+-------+
only showing top 6 rows



In order to view only certain
columns, we have to use the select method. Let us view only two columns
(age and mobile):

In [26]:
df.select('age','mobile').show(6)

+---+-------+
|age| mobile|
+---+-------+
| 32|   Vivo|
| 27|  Apple|
| 22|Samsung|
| 37|  Apple|
| 27|     MI|
| 27|   Oppo|
+---+-------+
only showing top 6 rows



**describe()** for analyzing the dataframe. It
returns the statistical measures for each column of the dataframe.
-For nonnumerical columns, it shows the
count and the min and max values, which are based on alphabetic order of
those fields and doesn’t signify any real meaning

In [27]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------+
|summary|           ratings|               age|        experience|            family|mobile|
+-------+------------------+------------------+------------------+------------------+------+
|  count|                33|                33|                33|                33|    33|
|   mean|3.5757575757575757|30.484848484848484|10.303030303030303|1.8181818181818181|  null|
| stddev|1.1188806636071336|  6.18527087180309| 6.770731351213326|1.8448330794164254|  null|
|    min|                 1|                22|               2.5|                 0| Apple|
|    max|                 5|                42|              23.0|                 5|  Vivo|
+-------+------------------+------------------+------------------+------------------+------+



###Adding a New Column


add a new column (age after 10 years) to our dataframe by using the age column

In [29]:
df.withColumn("age_after_10_yrs",(df["age"]+10)).show()
df = df.withColumn("age_after_10_yrs",(df["age"]+10))

+-------+---+----------+------+-------+----------------+
|ratings|age|experience|family| mobile|age_after_10_yrs|
+-------+---+----------+------+-------+----------------+
|      3| 32|       9.0|     3|   Vivo|              42|
|      3| 27|      13.0|     3|  Apple|              37|
|      4| 22|       2.5|     0|Samsung|              32|
|      4| 37|      16.5|     4|  Apple|              47|
|      5| 27|       9.0|     1|     MI|              37|
|      4| 27|       9.0|     0|   Oppo|              37|
|      5| 37|      23.0|     5|   Vivo|              47|
|      5| 37|      23.0|     5|Samsung|              47|
|      3| 22|       2.5|     0|  Apple|              32|
|      3| 27|       6.0|     0|     MI|              37|
|      2| 27|       6.0|     2|   Oppo|              37|
|      5| 27|       6.0|     2|Samsung|              37|
|      3| 37|      16.5|     5|  Apple|              47|
|      5| 27|       6.0|     0|     MI|              37|
|      4| 22|       6.0|     1|

Change the datatype of the age column from integer to double . use method cast and we need to import DoubleType from pyspark.types

In [43]:
from pyspark.sql.types import DoubleType
df = df.withColumn('age_double',df['age'].cast(DoubleType()))


In [45]:
df.show(10)

+-------+---+----------+------+-------+----------+
|ratings|age|experience|family| mobile|age_double|
+-------+---+----------+------+-------+----------+
|      3| 32|       9.0|     3|   Vivo|      32.0|
|      3| 27|      13.0|     3|  Apple|      27.0|
|      4| 22|       2.5|     0|Samsung|      22.0|
|      4| 37|      16.5|     4|  Apple|      37.0|
|      5| 27|       9.0|     1|     MI|      27.0|
|      4| 27|       9.0|     0|   Oppo|      27.0|
|      5| 37|      23.0|     5|   Vivo|      37.0|
|      5| 37|      23.0|     5|Samsung|      37.0|
|      3| 22|       2.5|     0|  Apple|      22.0|
|      3| 27|       6.0|     0|     MI|      27.0|
+-------+---+----------+------+-------+----------+
only showing top 10 rows



### Filtering Data

Filtering in PySpark is pretty straight-forward and can be
done using the filter function

We want to fetch the records for only ‘Vivo’ mobile

In [48]:
df.filter(df['mobile']=='Vivo').show()

+-------+---+----------+------+------+----------+
|ratings|age|experience|family|mobile|age_double|
+-------+---+----------+------+------+----------+
|      3| 32|       9.0|     3|  Vivo|      32.0|
|      5| 37|      23.0|     5|  Vivo|      37.0|
|      4| 37|       6.0|     0|  Vivo|      37.0|
|      5| 37|      13.0|     1|  Vivo|      37.0|
|      4| 37|       6.0|     0|  Vivo|      37.0|
+-------+---+----------+------+------+----------+



We want to view the age and ratings for people who use ‘Vivo’ mobile

In [54]:
df.filter(df['mobile']=='Vivo').select('age','ratings','mobile').show()

+---+-------+------+
|age|ratings|mobile|
+---+-------+------+
| 32|      3|  Vivo|
| 37|      5|  Vivo|
| 37|      4|  Vivo|
| 37|      5|  Vivo|
| 37|      4|  Vivo|
+---+-------+------+



We want to filter only ‘Vivo’ users and only those with experience of more than
10 years

In [58]:
df.filter((df['mobile']=="Vivo")&(df['experience']>10)).show()

+-------+---+----------+------+------+----------+
|ratings|age|experience|family|mobile|age_double|
+-------+---+----------+------+------+----------+
|      5| 37|      23.0|     5|  Vivo|      37.0|
|      5| 37|      13.0|     1|  Vivo|      37.0|
+-------+---+----------+------+------+----------+



Another approach to achieve the same
results 

In [61]:
df.filter(df['mobile']=='Vivo').filter(df['experience']>10).show()

+-------+---+----------+------+------+----------+
|ratings|age|experience|family|mobile|age_double|
+-------+---+----------+------+------+----------+
|      5| 37|      23.0|     5|  Vivo|      37.0|
|      5| 37|      13.0|     1|  Vivo|      37.0|
+-------+---+----------+------+------+----------+



### Distinct Values in Column

View the distinct values for the mobile
column in the dataframe **distinct()**

In [62]:
df.select('mobile').distinct().show()

+-------+
| mobile|
+-------+
|     MI|
|   Oppo|
|Samsung|
|   Vivo|
|  Apple|
+-------+



In [64]:
# count of distinct values in the column
df.select('mobile').distinct().count()

5

### Grouping Data


We are grouping all the records based on the categorical values in
the mobile column and counting the number of records for each category
using the count method

In [67]:
df.groupBy('mobile').count().show()

+-------+-----+
| mobile|count|
+-------+-----+
|     MI|    8|
|   Oppo|    7|
|Samsung|    6|
|   Vivo|    5|
|  Apple|    7|
+-------+-----+



 We can further refine these results by making use
of the orderBy method to sort them in a defined order

In [68]:
df.groupBy('mobile').count().orderBy('count',ascending=False).show()

+-------+-----+
| mobile|count|
+-------+-----+
|     MI|    8|
|   Oppo|    7|
|  Apple|    7|
|Samsung|    6|
|   Vivo|    5|
+-------+-----+



We can also apply the **groupBy** method to calculate statistical measures
such as mean value, sum, min, or max value for each category. 

In [69]:
df.groupBy('mobile').mean().show()

+-------+------------------+------------------+------------------+------------------+------------------+
| mobile|      avg(ratings)|          avg(age)|   avg(experience)|       avg(family)|   avg(age_double)|
+-------+------------------+------------------+------------------+------------------+------------------+
|     MI|               3.5|            30.125|           10.1875|             1.375|            30.125|
|   Oppo| 2.857142857142857|28.428571428571427|10.357142857142858|1.4285714285714286|28.428571428571427|
|Samsung| 4.166666666666667|28.666666666666668| 8.666666666666666|1.8333333333333333|28.666666666666668|
|   Vivo|               4.2|              36.0|              11.4|               1.8|              36.0|
|  Apple|3.4285714285714284|30.571428571428573|              11.0|2.7142857142857144|30.571428571428573|
+-------+------------------+------------------+------------------+------------------+------------------+



The mean method gives us the average of age, ratings, experience, and
family size columns for each mobile brand

Let us now view the min and max values of users data for every mobile
brand

In [70]:
df.groupBy('mobile').min().show()

+-------+------------+--------+---------------+-----------+---------------+
| mobile|min(ratings)|min(age)|min(experience)|min(family)|min(age_double)|
+-------+------------+--------+---------------+-----------+---------------+
|     MI|           1|      27|            2.5|          0|           27.0|
|   Oppo|           2|      22|            6.0|          0|           22.0|
|Samsung|           2|      22|            2.5|          0|           22.0|
|   Vivo|           3|      32|            6.0|          0|           32.0|
|  Apple|           3|      22|            2.5|          0|           22.0|
+-------+------------+--------+---------------+-----------+---------------+



In [71]:
df.groupBy('mobile').max().show()

+-------+------------+--------+---------------+-----------+---------------+
| mobile|max(ratings)|max(age)|max(experience)|max(family)|max(age_double)|
+-------+------------+--------+---------------+-----------+---------------+
|     MI|           5|      42|           23.0|          5|           42.0|
|   Oppo|           4|      42|           23.0|          2|           42.0|
|Samsung|           5|      37|           23.0|          5|           37.0|
|   Vivo|           5|      37|           23.0|          5|           37.0|
|  Apple|           4|      37|           16.5|          5|           37.0|
+-------+------------+--------+---------------+-----------+---------------+



### Aggregations

We can use the agg function as well to achieve the similar kinds of results
as above. Let’s use the agg function in PySpark for simply taking the sum of
total experience for each mobile brand.

In [72]:
df.groupBy('mobile').agg({'experience':'sum'}).show()

+-------+---------------+
| mobile|sum(experience)|
+-------+---------------+
|     MI|           81.5|
|   Oppo|           72.5|
|Samsung|           52.0|
|   Vivo|           57.0|
|  Apple|           77.0|
+-------+---------------+



## User-Defined Functions (UDFs)

UDFs are widely used in data processing to apply certain transformations
to the dataframe. There are two types of UDFs available in PySpark:
Conventional UDF and Pandas UDF. Pandas UDF are much more powerful
in terms of speed and processing time. 

In [74]:
#import udf from PySpark functions
from pyspark.sql.functions import udf
#Now we can apply basic UDF either by using a lambda or typical Python function

#### Traditional Python Function

In [76]:
def price_range(brand):
  if brand in ['Samsung','Apple']:
    return 'High Price'
  elif brand == 'Mi':
    return 'Mid Price'
  else:
    return 'Low Price'

Create a UDF (brand_udf) that uses this function
and also captures its datatype to apply this tranformation on the mobile
column of the dataframe

In [82]:
from pyspark.sql.types import  StringType
brand_udf = udf(price_range,StringType())

In the final step, we apply the udf(brand_udf) to the mobile column
of dataframe and create a new colum (price_range) with new values

In [83]:
df.withColumn('price_range',brand_udf(df['mobile'])).show()

+-------+---+----------+------+-------+----------+-----------+
|ratings|age|experience|family| mobile|age_double|price_range|
+-------+---+----------+------+-------+----------+-----------+
|      3| 32|       9.0|     3|   Vivo|      32.0|  Low Price|
|      3| 27|      13.0|     3|  Apple|      27.0| High Price|
|      4| 22|       2.5|     0|Samsung|      22.0| High Price|
|      4| 37|      16.5|     4|  Apple|      37.0| High Price|
|      5| 27|       9.0|     1|     MI|      27.0|  Low Price|
|      4| 27|       9.0|     0|   Oppo|      27.0|  Low Price|
|      5| 37|      23.0|     5|   Vivo|      37.0|  Low Price|
|      5| 37|      23.0|     5|Samsung|      37.0| High Price|
|      3| 22|       2.5|     0|  Apple|      22.0| High Price|
|      3| 27|       6.0|     0|     MI|      27.0|  Low Price|
|      2| 27|       6.0|     2|   Oppo|      27.0|  Low Price|
|      5| 27|       6.0|     2|Samsung|      27.0| High Price|
|      3| 37|      16.5|     5|  Apple|      37.0| High

### Using Lambda Function

We categorize the age columns into two groups (young, senior) based on
the age of the user.

In [86]:
age_udf = udf(lambda age: "young" if age<30 else "senior", StringType())
df.withColumn("age_group",age_udf(df['age'])).show(10)

+-------+---+----------+------+-------+----------+---------+
|ratings|age|experience|family| mobile|age_double|age_group|
+-------+---+----------+------+-------+----------+---------+
|      3| 32|       9.0|     3|   Vivo|      32.0|   senior|
|      3| 27|      13.0|     3|  Apple|      27.0|    young|
|      4| 22|       2.5|     0|Samsung|      22.0|    young|
|      4| 37|      16.5|     4|  Apple|      37.0|   senior|
|      5| 27|       9.0|     1|     MI|      27.0|    young|
|      4| 27|       9.0|     0|   Oppo|      27.0|    young|
|      5| 37|      23.0|     5|   Vivo|      37.0|   senior|
|      5| 37|      23.0|     5|Samsung|      37.0|   senior|
|      3| 22|       2.5|     0|  Apple|      22.0|    young|
|      3| 27|       6.0|     0|     MI|      27.0|    young|
+-------+---+----------+------+-------+----------+---------+
only showing top 10 rows



### Pandas UDF (Vectorized UDF)

Pandas UDFs are way faster and efficient
compared to their peers. There are two types of Pandas UDFs:

• Scalar

• GroupedMap

We have to first import pandas_udf from PySpark functions and apply it on any
particular column to be tranformed.

In [87]:
from pyspark.sql.functions import pandas_udf

Define a Python function that calculates the
number of years left in a user’s life assuming a life expectancy of 100 years.
It is a very simple calculation: we subtract the age of the user from 100
using a Python function

In [88]:
from pyspark.sql.types import IntegerType
def remainings_yrs(age):
  yrs_left = 100-age
  return yrs_left
length_udf = pandas_udf(remainings_yrs,IntegerType())
df.withColumn("Yrs_left",length_udf(df['age'])).show(10)

+-------+---+----------+------+-------+----------+--------+
|ratings|age|experience|family| mobile|age_double|Yrs_left|
+-------+---+----------+------+-------+----------+--------+
|      3| 32|       9.0|     3|   Vivo|      32.0|      68|
|      3| 27|      13.0|     3|  Apple|      27.0|      73|
|      4| 22|       2.5|     0|Samsung|      22.0|      78|
|      4| 37|      16.5|     4|  Apple|      37.0|      63|
|      5| 27|       9.0|     1|     MI|      27.0|      73|
|      4| 27|       9.0|     0|   Oppo|      27.0|      73|
|      5| 37|      23.0|     5|   Vivo|      37.0|      63|
|      5| 37|      23.0|     5|Samsung|      37.0|      63|
|      3| 22|       2.5|     0|  Apple|      22.0|      78|
|      3| 27|       6.0|     0|     MI|      27.0|      73|
+-------+---+----------+------+-------+----------+--------+
only showing top 10 rows



## Pandas UDF (Multiple Columns)

 We will create a new column that is simply the product of the ratings and experience columns. As usual, we define a Python function and calculate
the product of the two columns.

In [91]:
def prod(rating,exprience):
  x = rating * exprience
  return x
prod_udf = pandas_udf(prod,DoubleType())
df.withColumn('Product',prod_udf(df['ratings'],df['experience'])).show(10)

+-------+---+----------+------+-------+----------+-------+
|ratings|age|experience|family| mobile|age_double|Product|
+-------+---+----------+------+-------+----------+-------+
|      3| 32|       9.0|     3|   Vivo|      32.0|   27.0|
|      3| 27|      13.0|     3|  Apple|      27.0|   39.0|
|      4| 22|       2.5|     0|Samsung|      22.0|   10.0|
|      4| 37|      16.5|     4|  Apple|      37.0|   66.0|
|      5| 27|       9.0|     1|     MI|      27.0|   45.0|
|      4| 27|       9.0|     0|   Oppo|      27.0|   36.0|
|      5| 37|      23.0|     5|   Vivo|      37.0|  115.0|
|      5| 37|      23.0|     5|Samsung|      37.0|  115.0|
|      3| 22|       2.5|     0|  Apple|      22.0|    7.5|
|      3| 27|       6.0|     0|     MI|      27.0|   18.0|
+-------+---+----------+------+-------+----------+-------+
only showing top 10 rows



## Drop Duplicate Values

We can use the **dropDuplicates()** method in order to remove the duplicate
records from the dataframe. The total number of records in this dataframe
are 33, but it also contains 7 duplicate records, which can easily be confirmed
by droping those duplicate records as we are left with only 26 rows.

In [ ]:
df.count()

In [92]:

df = df.dropDuplicates()
df.count()

26

## Delete Column

Remove the mobile column from the
dataframe bypass it as an argument inside the drop function

In [96]:
df_new = df.drop('mobile')
df_new.show(10)

+-------+---+----------+------+----------+
|ratings|age|experience|family|age_double|
+-------+---+----------+------+----------+
|      4| 27|       6.0|     1|      27.0|
|      3| 27|       6.0|     0|      27.0|
|      3| 37|      16.5|     5|      37.0|
|      3| 22|       2.5|     0|      22.0|
|      5| 27|       6.0|     2|      27.0|
|      5| 37|      23.0|     5|      37.0|
|      5| 27|       9.0|     1|      27.0|
|      2| 27|       9.0|     2|      27.0|
|      4| 37|      16.5|     4|      37.0|
|      5| 22|       2.5|     0|      22.0|
+-------+---+----------+------+----------+
only showing top 10 rows



## Writing Data

 **Csv**
- save it back in the original csv format as single file, we can use
the **coalesce** function in spark

In [97]:
pwd

'/content'

In [98]:
write_Loca = '/content/df_csv'
df.coalesce(1).write.format("csv").option('header','True').save(write_Loca)

**Parquet**

If the dataset is huge and involves a lot of columns, we can choose to
compress it and convert it into a parquet file format. It reduces the overall
size of the data and optimizes the performance while processing data
because it works on subsets of required columns instead of the entire data

In [99]:
parquet_uri = '/content/df_parquet'
df.write.format('parquet').save(parquet_uri)